In [1]:
from google.colab import drive
drive.mount('/content/drive')
#klik the link ==> choose your gmail account ==> click allow ==> copy the key and paste in this field

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
#path for the data folder
HOME_DIR = '/content/drive/My Drive/Colab Notebooks/DSI Data/'

In [4]:
# import important library
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
import pickle
%matplotlib inline

In [11]:
# Open cs file with pandas
df = pd.read_csv(HOME_DIR+'data/ecom_op.csv', header = None)
df = df[[0,1]]
df.columns = ['tweet', 'label']
df.head()
# In NLP a tweet called as document, then the collection of document called as corpus
# so df.iloc(0)[0] called as document

,tweet,label
0,@firstlogistics_ @bliblidotcom @bliblicare @yl...,-1
1,@bliblidotcom @bliblicare tidak ada link yg pa...,-1
2,"@bliblidotcom min ga relavan twittnya, sekaran...",-1
3,@bcaklikpay @ferdian08 @bliblidotcom jawaban p...,-1
4,waah harus hati2 nih.. kl yg ga teliti bs ke p...,-1


In [13]:
#the number of document in every label
print(df.shape)
print(df.label.value_counts())

(3144, 2)
 0    2194
 1     528
-1     422
Name: label, dtype: int64


In [ ]:
#look at the data before starting preprocessing
df.tweet.head(10).values

array(['@firstlogistics_ @bliblidotcom @bliblicare @ylki_id rasanya kecewa bgt sm jasa logistik satu ini, gak ada niat baik. ',
       '@bliblidotcom @bliblicare tidak ada link yg pasti produknya speaker niko harga 160rbuan kirim ke manyar gresik masa kena ongkir 90rb ',
       '@bliblidotcom min ga relavan twittnya, sekarang kan tanggal 19 april .. berarti ini twit dari masa lalu ya min? ',
       '@bcaklikpay @ferdian08 @bliblidotcom jawaban pun klise, besok dicoba lagi bla bla bla.. klikpay emg ga pnh belajar dr pglmn kmrn2.. ',
       'waah harus hati2 nih.. kl yg ga teliti bs ke potong harga normal.. parah jg ya @bcaklikpay dan @bliblidotcom ',
       '@bcaklikpay @ferdian08 @bliblidotcom ane jg kena php, di chart dpt disc 65rb, proses ke klikpay kena harga normal.. ',
       '@bcaklikpay @bliblidotcom bkn mslh coba lagi, tp krn kegoblokan sistem @bliblidotcom jd ga dpt.. ',
       '@bliblidotcom daftar merchant kok gak ada respon ya sejak kemarin? ',
       '@bliblicare no. pesan

In [14]:
#split the data set into train and test with test_size = 0.2 (equal with 80% train and 20% testing)
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(df.tweet, df.label, test_size=0.2)

In [15]:
#load stopword data from file
stp = open(HOME_DIR+'data/twitter_stp.dic')
stp_words = []
items = stp.readline()
while items != "":
  stp_words.append(items[:-1]) #just read word without /n
  items = stp.readline()

In [16]:
len(stp_words)

987

In [17]:
# feature extraction 
# min_df (minimal number of vocab), max_df(max number of voceb)
# regez (token_pattern), just take character
vect = TfidfVectorizer(min_df=0, max_df=0.9, stop_words=stp_words,
                       token_pattern = '\\b[a-zA-Z][a-zA-Z][a-zA-Z]+\\b')

In [18]:
#Model for classifier, arrange the hyperparameter to get the best performance
from sklearn.ensemble import RandomForestClassifier
# machine learning model
# alpha for smoothing 
# model = MultinomialNB(alpha=0.3)
model = RandomForestClassifier(min_samples_leaf=1, min_samples_split=3,
                                        min_weight_fraction_leaf=0.0,
                                        n_estimators=125)

In [19]:
# create pipeline
pipeline = Pipeline([
                     ('vect',vect),
                     ('clf',model)
])

In [20]:
#train the model
pipeline.fit(x_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['olah', 'rasa'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


Pipeline(memory=None,
         steps=[('vect',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=0.9, max_features=None,
                                 min_df=0, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=['a', 'ada', 'adalah', 'ah', 'akan',
                                             'aku', 'all', 'an', 'and', 'apa...
                 RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                        class_weight=None, criterion='gini',
                                        max_depth=None, max_features='auto',
                                        max_leaf_nodes=None, max_samples=None,
             

In [21]:
# predict the label of x_test data
y_pred = pipeline.predict(x_test)

In [ ]:
#evaluation the model (use confusion matrix)
print(classification_report(y_pred, y_test))

              precision    recall  f1-score   support

          -1       0.35      0.83      0.50        35
           0       0.96      0.79      0.87       523
           1       0.50      0.85      0.63        71

    accuracy                           0.79       629
   macro avg       0.60      0.82      0.66       629
weighted avg       0.88      0.79      0.82       629



In [24]:
#accuracy of our model
from sklearn import metrics
print(metrics.accuracy_score(y_test, y_pred))

0.7837837837837838


In [22]:
#try with new data
new_tweet = ["rasanya kecewa bgt sm jasa logistik satu ini, gak ada niat baik",
             "Proses Transaksi sangat cepat dan saya puas sekali"]
pipeline.predict(new_tweet)

array([-1,  1])

In [23]:
# dump pipeline to pickle
pickle.dump(pipeline,open(HOME_DIR+'gridsearchvalue.pkl', 'wb'))

In [ ]:
#if you want to get the best parameter for your model, do hyperparameter tuning and use the best parameter to your model
#grid search cv (your can use gridsearchcv or randomizesearchcv)
from sklearn.model_selection import GridSearchCV

#karenda didalam clf, kasih clf untuk identifier
params = {'vect__min_df':[0,0.05,0.1],
          'vect__max_df':[0.9, 0.95, 0.99, 1],
          'clf__min_samples_leaf':[1,2,3,4,5],
          'clf__min_samples_split':[2,3],
          'clf__min_weight_fraction_leaf':[0.0,0.05,0.1,0.15],
          'clf__n_estimators':[100,125,150]}

In [ ]:
grid = GridSearchCV(pipeline,
                    params,
                    n_jobs = 5,
                    cv=5)

In [ ]:
#train your model with gridsearchcv
grid.fit(x_train,y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['olah', 'rasa'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        TfidfVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.float64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=0.99,
                                                        max_features=None,
                                                        min_df=0.005,
                                                        ngram_range=(1, 1),
                                                     

In [ ]:
#this is the best parameter from gridsearcch
#try to train your model above using this parameter
grid.best_estimator_

Pipeline(memory=None,
         steps=[('vect',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=0.9, max_features=None,
                                 min_df=0, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=['a', 'ada', 'adalah', 'ah', 'akan',
                                             'aku', 'all', 'an', 'and', 'apa...
                 RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                        class_weight=None, criterion='gini',
                                        max_depth=None, max_features='auto',
                                        max_leaf_nodes=None, max_samples=None,
             

In [ ]:
#precisson : akurasi orang lulus dari semua orang2 lulus
#recall : akurasi tidak lulus dari orang2 tidak lulus